In [10]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv

# Ouvrir l'URL et analyser le contenu HTML
html = urlopen('https://www.ebay.fr/b/Telephones-mobiles/9355/bn_16576651')
bs = BeautifulSoup(html.read(), 'html.parser')

# Scrapper les articles de Téléphones mobiles
info = bs.find_all('div', {'class': 's-item__info clearfix'})

# Ouvrir un fichier CSV pour écrire les données
with open('smartphones.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Écrire les en-têtes
    writer.writerow(['Article','Statut','Prix (€)', 'Prix de vente initial (€)', 'Frais de livraison (€)', 'Suivi par (personnes)', 'Vendus', 'Lien du produit'])

    # Extraire et écrire les informations de chaque article
    for item in info:
        title = item.find('h3', {'class': 's-item__title'}).get_text()
        price = item.find('span', {'class': 's-item__price'}).get_text().replace('EUR', '').strip()
        
        initial_price_element = item.find('span', {'class': 'STRIKETHROUGH'})
        if initial_price_element:
            initial_price = initial_price_element.get_text().replace('EUR', '').strip()
        else:
            initial_price = None

        text_element = item.find('span', {'class': 'NEGATIVE'})
        if text_element:
            text = text_element.get_text()
        else:
            text = ""

        link = item.find('a', {'class': 's-item__link'})['href']
        frais_livraison_element = item.find('span', {'class': 's-item__shipping s-item__logisticsCost'})
        if frais_livraison_element:
            frais_livraison = frais_livraison_element.get_text().replace('de frais de livraison', '').replace('EUR', '').strip()
        else:
            frais_livraison = "0"

        status_element = item.find('div', {'class': 's-item__subtitle'})
        if status_element:
            status = status_element.get_text()
        else:
            status = ""

        followed = 0
        vendus = 0
        if "Suivi par" in text:
            followed = ''.join([word for word in text.split() if word.isdigit()])
        elif "Il n'en reste plus qu'un" in text:
            followed = 1
        if "vendus" in text:
            vendus = ''.join([word for word in text.split() if word.isdigit()])

        # Écrire les données dans le fichier CSV
        writer.writerow([title,status, price, initial_price, frais_livraison, followed, vendus, link])

        # Afficher les données de chaque article scrapé
        print("Article :", title)
        print("Statut :", status)
        print("Prix (€) :", price)
        if initial_price:
            print("Prix de vente initial (€) :", initial_price)
        print("Frais de livraison (€) :", frais_livraison)
        if followed:
            print("Suivi par (personnes) :", followed)
        if vendus:
            print("Vendus :", vendus)
        print("Lien du produit :", link)
        print("\n")


Article : 【Lowest price】Apple iPhone11/iPhoneXS/iPhoneXR/iPhone8Plus/iPhone8/iPhone6
Statut : Professionnel
Prix (€) : 60,80  à 239,42
Frais de livraison (€) : 6,50
Lien du produit : https://www.ebay.fr/itm/404955806035?hash=item5e493f3953:g:0t4AAOSwf5BmNHZZ&amdata=enc%3AAQAJAAAA0E78rei%2FhxcYO6JMpNXFycw5QGq8DxIx4Z2t7KJl1D9%2Ff3g3T7ipNy711GPYVGqjI4Ct2oZAxoIayPxJdFAb0Kwx3NQ%2FWIAbz0IKfU6ao2f3D4dXc85Lv0oHpQ%2Fr9rFivMCLrWzDRew1kqvsoODLGmLI5V4Am4YBMXVVq9oACZjGHq581cUumByNBwwkBOu7WhT7B8Q6Eg5JVIK1ytIEhKghfCndP7xN4zhuBvasZ7tpzc14BYEKq9e2V%2BWnvr3nK4EahUL8kajUn%2FuWcbzgUqs%3D%7Ctkp%3ABFBMxrzs_IJk&var=675000066648


Article : Smartphone i15 Pro Max 7,3 pouces double SIM global flambant neuf débloqué téléphone Android
Statut : Professionnel
Prix (€) : 106,66  à 154,07
Frais de livraison (€) : 5,91
Suivi par (personnes) : 20
Lien du produit : https://www.ebay.fr/itm/285833999867?hash=item428d08b5fb:g:5u8AAOSwV-hlQ2v8&amdata=enc%3AAQAJAAAA0BCrEEm%2FcfRBPArJL1p9m8Xlqv7AMcxek9e49LSsApma%2B5ID2VkPx%2